## 1. Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de sklearn para transformaciones
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
    Normalizer,
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder
)

# Para comparar modelos
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

import warnings
warnings.filterwarnings('ignore')

## 2. Cargar y Preparar los Datos

In [ ]:
# Cargar el dataset del Titanic
df = pd.read_csv('Titanic-Dataset.csv')

print("Dimensiones del dataset:", df.shape)
df.head()

In [ ]:
# Información general
df.info()

In [ ]:
# Identificar tipos de variables
print("Variables numéricas:")
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(numeric_cols)

print("\nVariables categóricas:")
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(categorical_cols)

---

# PARTE I: NORMALIZACIÓN Y ESCALADO

## ¿Por qué normalizar?

Muchos algoritmos de Machine Learning son sensibles a la escala de las características:

- **Regresión Logística**: Converge más rápido con datos escalados
- **SVM**: Muy sensible a la escala
- **K-Means**: Usa distancias euclideas
- **Redes Neuronales**: Entrenan mejor con datos normalizados
- **KNN**: Basado en distancias

**Algoritmos que NO requieren escalado**: Árboles de decisión, Random Forest, XGBoost

## 3. Análisis de Escalas Originales

In [ ]:
# Seleccionar variables numéricas para análisis
features_for_scaling = ['Age', 'Fare', 'SibSp', 'Parch']

# Estadísticas descriptivas
print("Estadísticas de las variables originales:")
print("="*70)
df[features_for_scaling].describe()

In [ ]:
# Visualizar las diferentes escalas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, col in enumerate(features_for_scaling):
    axes[idx].hist(df[col].dropna(), bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribución de {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Valor')
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', label=f'Media: {df[col].mean():.2f}')
    axes[idx].legend()

plt.tight_layout()
plt.show()

print("\n⚠️  Observa las diferentes escalas:")
print(f"   - Age: rango aproximado 0-80")
print(f"   - Fare: rango aproximado 0-500")
print(f"   - SibSp y Parch: rango 0-8")

In [ ]:
# Comparación visual de escalas
df_clean = df[features_for_scaling].dropna()

plt.figure(figsize=(12, 6))
plt.boxplot([df_clean[col] for col in features_for_scaling], labels=features_for_scaling)
plt.title('Comparación de Escalas - Datos Originales', fontsize=14, fontweight='bold')
plt.ylabel('Valor')
plt.grid(True, alpha=0.3)
plt.show()

print("\n📊 Los boxplots muestran que 'Fare' tiene una escala mucho mayor que las demás variables.")

## 4. Método 1: StandardScaler (Estandarización)

**Fórmula**: $z = \frac{x - \mu}{\sigma}$

Donde:
- $x$ = valor original
- $\mu$ = media
- $\sigma$ = desviación estándar

**Resultado**: Media = 0, Desviación estándar = 1

**Cuándo usar**:
- Cuando los datos siguen una distribución normal o aproximadamente normal
- Para algoritmos que asumen distribución normal (Regresión Logística, SVM)
- Cuando hay outliers moderados

In [ ]:
# Preparar datos (eliminar nulos para este ejemplo)
df_clean = df[features_for_scaling].dropna().copy()

# Aplicar StandardScaler
scaler_standard = StandardScaler()
df_standard = pd.DataFrame(
    scaler_standard.fit_transform(df_clean),
    columns=[f'{col}_standard' for col in features_for_scaling],
    index=df_clean.index
)

print("Estadísticas después de StandardScaler:")
print("="*70)
print(df_standard.describe())
print("\n✓ Media ≈ 0, Desviación estándar ≈ 1")

In [ ]:
# Visualizar el efecto de StandardScaler
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, col in enumerate(features_for_scaling):
    axes[idx].hist(df_standard[f'{col}_standard'], bins=30, edgecolor='black', alpha=0.7, color='green')
    axes[idx].set_title(f'{col} - StandardScaler', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Valor Estandarizado')
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].axvline(0, color='red', linestyle='--', label='Media = 0')
    axes[idx].legend()

plt.tight_layout()
plt.show()

## 5. Método 2: MinMaxScaler (Normalización)

**Fórmula**: $x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$

**Resultado**: Valores en el rango [0, 1]

**Cuándo usar**:
- Cuando necesitas datos en un rango específico (0-1)
- Para redes neuronales
- Cuando NO hay outliers extremos
- Para algoritmos que esperan datos en un rango fijo

In [ ]:
# Aplicar MinMaxScaler
scaler_minmax = MinMaxScaler()
df_minmax = pd.DataFrame(
    scaler_minmax.fit_transform(df_clean),
    columns=[f'{col}_minmax' for col in features_for_scaling],
    index=df_clean.index
)

print("Estadísticas después de MinMaxScaler:")
print("="*70)
print(df_minmax.describe())
print("\n✓ Valores entre 0 y 1")

In [ ]:
# Visualizar el efecto de MinMaxScaler
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, col in enumerate(features_for_scaling):
    axes[idx].hist(df_minmax[f'{col}_minmax'], bins=30, edgecolor='black', alpha=0.7, color='blue')
    axes[idx].set_title(f'{col} - MinMaxScaler', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Valor Normalizado [0, 1]')
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].set_xlim(-0.1, 1.1)

plt.tight_layout()
plt.show()

## 6. Método 3: RobustScaler (Escalado Robusto)

**Fórmula**: $x_{scaled} = \frac{x - Q_2}{Q_3 - Q_1}$

Donde:
- $Q_2$ = mediana
- $Q_1$ = percentil 25
- $Q_3$ = percentil 75

**Cuándo usar**:
- **Cuando hay OUTLIERS**
- Usa la mediana en lugar de la media
- Usa el rango intercuartílico (IQR) en lugar de la desviación estándar
- Más robusto a valores extremos

In [ ]:
# Aplicar RobustScaler
scaler_robust = RobustScaler()
df_robust = pd.DataFrame(
    scaler_robust.fit_transform(df_clean),
    columns=[f'{col}_robust' for col in features_for_scaling],
    index=df_clean.index
)

print("Estadísticas después de RobustScaler:")
print("="*70)
print(df_robust.describe())
print("\n✓ Centrado en la mediana, escalado por IQR")

In [ ]:
# Comparación de los tres métodos para la variable 'Fare' (que tiene outliers)
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Original
axes[0].hist(df_clean['Fare'], bins=30, edgecolor='black', alpha=0.7, color='gray')
axes[0].set_title('Original', fontweight='bold')
axes[0].set_xlabel('Fare')

# StandardScaler
axes[1].hist(df_standard['Fare_standard'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('StandardScaler', fontweight='bold')
axes[1].set_xlabel('Fare Estandarizado')

# MinMaxScaler
axes[2].hist(df_minmax['Fare_minmax'], bins=30, edgecolor='black', alpha=0.7, color='blue')
axes[2].set_title('MinMaxScaler', fontweight='bold')
axes[2].set_xlabel('Fare Normalizado')

# RobustScaler
axes[3].hist(df_robust['Fare_robust'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[3].set_title('RobustScaler', fontweight='bold')
axes[3].set_xlabel('Fare Robusto')

plt.tight_layout()
plt.show()

print("\n💡 Observa cómo RobustScaler maneja mejor los outliers en 'Fare'")

## 7. Comparación Lado a Lado de Todos los Métodos

In [ ]:
# Crear tabla comparativa
comparison = pd.DataFrame({
    'Métrica': ['Media', 'Std', 'Min', 'Max', 'Q1', 'Q2 (Mediana)', 'Q3'],
    'Original': [
        df_clean['Fare'].mean(),
        df_clean['Fare'].std(),
        df_clean['Fare'].min(),
        df_clean['Fare'].max(),
        df_clean['Fare'].quantile(0.25),
        df_clean['Fare'].quantile(0.50),
        df_clean['Fare'].quantile(0.75)
    ],
    'StandardScaler': [
        df_standard['Fare_standard'].mean(),
        df_standard['Fare_standard'].std(),
        df_standard['Fare_standard'].min(),
        df_standard['Fare_standard'].max(),
        df_standard['Fare_standard'].quantile(0.25),
        df_standard['Fare_standard'].quantile(0.50),
        df_standard['Fare_standard'].quantile(0.75)
    ],
    'MinMaxScaler': [
        df_minmax['Fare_minmax'].mean(),
        df_minmax['Fare_minmax'].std(),
        df_minmax['Fare_minmax'].min(),
        df_minmax['Fare_minmax'].max(),
        df_minmax['Fare_minmax'].quantile(0.25),
        df_minmax['Fare_minmax'].quantile(0.50),
        df_minmax['Fare_minmax'].quantile(0.75)
    ],
    'RobustScaler': [
        df_robust['Fare_robust'].mean(),
        df_robust['Fare_robust'].std(),
        df_robust['Fare_robust'].min(),
        df_robust['Fare_robust'].max(),
        df_robust['Fare_robust'].quantile(0.25),
        df_robust['Fare_robust'].quantile(0.50),
        df_robust['Fare_robust'].quantile(0.75)
    ]
})

print("Comparación de Métodos de Escalado para 'Fare':")
print("="*90)
print(comparison.to_string(index=False))

## 8. Impacto en el Rendimiento del Modelo

In [ ]:
# Preparar datos para modelado
df_model = df[['Age', 'Fare', 'SibSp', 'Parch', 'Survived']].dropna()

X = df_model[['Age', 'Fare', 'SibSp', 'Parch']]
y = df_model['Survived']

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Función para entrenar y evaluar
def train_evaluate(X_train, X_test, y_train, y_test, scaler_name="Sin escalar"):
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Evaluar sin escalar
acc_no_scaling = train_evaluate(X_train, X_test, y_train, y_test, "Sin escalar")

# Evaluar con StandardScaler
scaler_std = StandardScaler()
X_train_std = scaler_std.fit_transform(X_train)
X_test_std = scaler_std.transform(X_test)
acc_standard = train_evaluate(X_train_std, X_test_std, y_train, y_test, "StandardScaler")

# Evaluar con MinMaxScaler
scaler_mm = MinMaxScaler()
X_train_mm = scaler_mm.fit_transform(X_train)
X_test_mm = scaler_mm.transform(X_test)
acc_minmax = train_evaluate(X_train_mm, X_test_mm, y_train, y_test, "MinMaxScaler")

# Evaluar con RobustScaler
scaler_rb = RobustScaler()
X_train_rb = scaler_rb.fit_transform(X_train)
X_test_rb = scaler_rb.transform(X_test)
acc_robust = train_evaluate(X_train_rb, X_test_rb, y_train, y_test, "RobustScaler")

# Mostrar resultados
results = pd.DataFrame({
    'Método': ['Sin escalar', 'StandardScaler', 'MinMaxScaler', 'RobustScaler'],
    'Accuracy': [acc_no_scaling, acc_standard, acc_minmax, acc_robust]
})

print("\nComparación de Rendimiento (Regresión Logística):")
print("="*60)
print(results.to_string(index=False))

In [ ]:
# Visualizar comparación
plt.figure(figsize=(10, 6))
bars = plt.bar(results['Método'], results['Accuracy'], color=['gray', 'green', 'blue', 'orange'], alpha=0.7, edgecolor='black')
plt.title('Comparación de Accuracy según Método de Escalado', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Método de Escalado', fontsize=12)
plt.ylim(0.6, 0.75)
plt.grid(axis='y', alpha=0.3)

# Añadir valores sobre las barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}',
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

---

# PARTE II: CODIFICACIÓN CATEGÓRICA

## ¿Por qué codificar variables categóricas?

Los algoritmos de Machine Learning trabajan con números, no con texto. Necesitamos convertir variables categóricas en numéricas.

**Tipos de variables categóricas**:
1. **Nominales**: Sin orden (Ej: Color, Ciudad, Sexo)
2. **Ordinales**: Con orden (Ej: Educación, Satisfacción, Clase del Titanic)

## 9. Exploración de Variables Categóricas

In [ ]:
# Variables categóricas en el dataset
categorical_features = ['Sex', 'Embarked', 'Pclass']

for col in categorical_features:
    print(f"\n{'='*60}")
    print(f"Variable: {col}")
    print(f"{'='*60}")
    print(df[col].value_counts())
    print(f"\nNúmero de categorías únicas: {df[col].nunique()}")
    print(f"Valores nulos: {df[col].isnull().sum()}")

In [ ]:
# Visualizar distribución de variables categóricas
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, col in enumerate(categorical_features):
    df[col].value_counts().plot(kind='bar', ax=axes[idx], color='skyblue', edgecolor='black')
    axes[idx].set_title(f'Distribución de {col}', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 10. Método 1: Label Encoding

Asigna un número entero a cada categoría.

**Ejemplo**: male=0, female=1

**Cuándo usar**:
- ✅ Variables ordinales (con orden natural)
- ✅ Variables objetivo (target)
- ✅ Árboles de decisión

**Cuándo NO usar**:
- ❌ Variables nominales en modelos lineales (implica orden artificial)
- ❌ Cuando el orden no tiene sentido

In [ ]:
# Label Encoding para 'Sex'
df_label = df.copy()

label_encoder = LabelEncoder()
df_label['Sex_Encoded'] = label_encoder.fit_transform(df_label['Sex'])

print("Label Encoding de 'Sex':")
print("="*50)
print(df_label[['Sex', 'Sex_Encoded']].drop_duplicates().sort_values('Sex_Encoded'))

print("\nMapeo:")
for i, label in enumerate(label_encoder.classes_):
    print(f"  {label} → {i}")

In [ ]:
# Label Encoding para 'Embarked'
df_label['Embarked_Encoded'] = label_encoder.fit_transform(df_label['Embarked'].fillna('Unknown'))

print("Label Encoding de 'Embarked':")
print("="*50)
print(df_label[['Embarked', 'Embarked_Encoded']].drop_duplicates().sort_values('Embarked_Encoded'))

## 11. Método 2: Ordinal Encoding

Similar a Label Encoding, pero permite especificar el orden manualmente.

**Cuándo usar**:
- Variables ordinales donde el orden es importante
- Ejemplo: Educación (Primaria < Secundaria < Universidad)
- Ejemplo en Titanic: Pclass (1ª clase > 2ª clase > 3ª clase)

In [ ]:
# Ordinal Encoding para 'Pclass' (tiene orden: 1 > 2 > 3)
ordinal_encoder = OrdinalEncoder(
    categories=[[1, 2, 3]]  # Especificamos el orden
)

df_ordinal = df.copy()
df_ordinal['Pclass_Ordinal'] = ordinal_encoder.fit_transform(df_ordinal[['Pclass']])

print("Ordinal Encoding de 'Pclass':")
print("="*50)
print(df_ordinal[['Pclass', 'Pclass_Ordinal']].drop_duplicates().sort_values('Pclass'))

print("\n💡 Pclass ya es numérico, pero podemos usarlo como ejemplo de variable ordinal.")

## 12. Método 3: One-Hot Encoding

Crea una columna binaria (0/1) para cada categoría.

**Ejemplo**: 
- Embarked=S → Embarked_S=1, Embarked_C=0, Embarked_Q=0
- Embarked=C → Embarked_S=0, Embarked_C=1, Embarked_Q=0

**Cuándo usar**:
- ✅ Variables nominales (sin orden)
- ✅ Modelos lineales, SVM, Redes Neuronales
- ✅ Cuando las categorías no tienen relación ordinal

**Desventajas**:
- ❌ Aumenta la dimensionalidad (muchas columnas)
- ❌ Problema con alta cardinalidad (muchas categorías únicas)

In [ ]:
# One-Hot Encoding con pandas get_dummies
df_onehot = df[['Sex', 'Embarked', 'Pclass']].copy()
df_onehot_encoded = pd.get_dummies(df_onehot, columns=['Sex', 'Embarked'], prefix=['Sex', 'Embarked'])

print("Antes de One-Hot Encoding:")
print("="*50)
print(df_onehot.head())

print("\n\nDespués de One-Hot Encoding:")
print("="*50)
print(df_onehot_encoded.head())

print(f"\nColumnas originales: {df_onehot.shape[1]}")
print(f"Columnas después de One-Hot: {df_onehot_encoded.shape[1]}")

In [ ]:
# One-Hot Encoding con sklearn (más control)
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' evita multicolinealidad

# Codificar 'Embarked'
embarked_encoded = onehot_encoder.fit_transform(df[['Embarked']].fillna('Unknown'))
embarked_feature_names = onehot_encoder.get_feature_names_out(['Embarked'])

df_onehot_sklearn = pd.DataFrame(
    embarked_encoded,
    columns=embarked_feature_names,
    index=df.index
)

print("One-Hot Encoding de 'Embarked' (con drop='first'):")
print("="*60)
print(df_onehot_sklearn.head(10))

print("\n💡 Con drop='first', eliminamos una columna para evitar multicolinealidad.")
print("   (Si todas las otras son 0, sabemos que es la categoría eliminada)")

## 13. Comparación de Métodos de Codificación

In [ ]:
# Crear DataFrame comparativo
sample_data = pd.DataFrame({
    'Original': ['C', 'S', 'Q', 'S', 'C'],
})

# Label Encoding
le = LabelEncoder()
sample_data['Label_Encoding'] = le.fit_transform(sample_data['Original'])

# One-Hot Encoding
onehot_sample = pd.get_dummies(sample_data['Original'], prefix='OneHot')

# Combinar
comparison_df = pd.concat([sample_data, onehot_sample], axis=1)

print("Comparación de Métodos de Codificación:")
print("="*70)
print(comparison_df)

## 14. Impacto en el Rendimiento del Modelo

In [ ]:
# Preparar datos con diferentes codificaciones
df_model_cat = df[['Age', 'Fare', 'Sex', 'Embarked', 'Pclass', 'Survived']].copy()
df_model_cat = df_model_cat.dropna()

# 1. Con Label Encoding
df_label_model = df_model_cat.copy()
df_label_model['Sex'] = LabelEncoder().fit_transform(df_label_model['Sex'])
df_label_model['Embarked'] = LabelEncoder().fit_transform(df_label_model['Embarked'])

X_label = df_label_model.drop('Survived', axis=1)
y_label = df_label_model['Survived']

X_train_label, X_test_label, y_train_label, y_test_label = train_test_split(
    X_label, y_label, test_size=0.2, random_state=42
)

# Escalar y entrenar
scaler = StandardScaler()
X_train_label_scaled = scaler.fit_transform(X_train_label)
X_test_label_scaled = scaler.transform(X_test_label)

model_label = LogisticRegression(max_iter=1000, random_state=42)
model_label.fit(X_train_label_scaled, y_train_label)
acc_label = accuracy_score(y_test_label, model_label.predict(X_test_label_scaled))

print(f"Accuracy con Label Encoding: {acc_label:.4f}")

In [ ]:
# 2. Con One-Hot Encoding
df_onehot_model = df_model_cat.copy()
df_onehot_model = pd.get_dummies(df_onehot_model, columns=['Sex', 'Embarked'], drop_first=True)

X_onehot = df_onehot_model.drop('Survived', axis=1)
y_onehot = df_onehot_model['Survived']

X_train_onehot, X_test_onehot, y_train_onehot, y_test_onehot = train_test_split(
    X_onehot, y_onehot, test_size=0.2, random_state=42
)

# Escalar y entrenar
scaler_oh = StandardScaler()
X_train_onehot_scaled = scaler_oh.fit_transform(X_train_onehot)
X_test_onehot_scaled = scaler_oh.transform(X_test_onehot)

model_onehot = LogisticRegression(max_iter=1000, random_state=42)
model_onehot.fit(X_train_onehot_scaled, y_train_onehot)
acc_onehot = accuracy_score(y_test_onehot, model_onehot.predict(X_test_onehot_scaled))

print(f"Accuracy con One-Hot Encoding: {acc_onehot:.4f}")

In [ ]:
# Comparación final
encoding_results = pd.DataFrame({
    'Método': ['Label Encoding', 'One-Hot Encoding'],
    'Accuracy': [acc_label, acc_onehot],
    'Nº Features': [X_train_label.shape[1], X_train_onehot.shape[1]]
})

print("\n" + "="*70)
print("COMPARACIÓN DE MÉTODOS DE CODIFICACIÓN")
print("="*70)
print(encoding_results.to_string(index=False))

print("\n💡 One-Hot Encoding generalmente funciona mejor para variables nominales,")
print("   aunque aumenta el número de características.")

## 15. Guía de Decisión: ¿Qué Método Usar?

In [ ]:
# Crear tabla de decisión
decision_guide = pd.DataFrame({
    'Escenario': [
        'Variable nominal + Modelo lineal',
        'Variable nominal + Árbol de decisión',
        'Variable ordinal',
        'Muchas categorías (>10)',
        'Target variable (y)',
        'Datos con outliers',
        'Redes Neuronales',
        'Sin outliers + Modelo lineal'
    ],
    'Normalización': [
        'StandardScaler',
        'No necesario',
        'StandardScaler',
        'RobustScaler',
        'No aplicable',
        'RobustScaler',
        'MinMaxScaler',
        'StandardScaler o MinMax'
    ],
    'Codificación': [
        'One-Hot Encoding',
        'Label Encoding',
        'Ordinal Encoding',
        'Target/Frequency Encoding',
        'Label Encoding',
        'No aplicable',
        'One-Hot Encoding',
        'One-Hot Encoding'
    ]
})

print("\n" + "="*90)
print("GUÍA DE DECISIÓN: NORMALIZACIÓN Y CODIFICACIÓN")
print("="*90)
print(decision_guide.to_string(index=False))

## 16. Ejemplo Completo: Pipeline de Transformación

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Definir características
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch']
categorical_features = ['Sex', 'Embarked', 'Pclass']

# Pipeline para variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())  # Usamos RobustScaler por los outliers en Fare
])

# Pipeline para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Combinar con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Pipeline completo
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Preparar datos
X = df[numeric_features + categorical_features]
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar
full_pipeline.fit(X_train, y_train)

# Evaluar
acc_pipeline = full_pipeline.score(X_test, y_test)

print("\n" + "="*70)
print("PIPELINE COMPLETO DE TRANSFORMACIÓN")
print("="*70)
print(f"\nAccuracy: {acc_pipeline:.4f}")
print(f"\n✓ Variables numéricas: Imputadas con mediana + RobustScaler")
print(f"✓ Variables categóricas: Imputadas + One-Hot Encoding")
print(f"✓ Modelo: Regresión Logística")

## 17. Resumen y Mejores Prácticas

### 📊 Normalización/Escalado:

| Método | Fórmula | Cuándo usar | Resultado |
|--------|---------|-------------|------------|
| **StandardScaler** | $(x - \mu) / \sigma$ | Datos normales, sin outliers extremos | Media=0, Std=1 |
| **MinMaxScaler** | $(x - min) / (max - min)$ | Necesitas rango [0,1], sin outliers | Rango [0, 1] |
| **RobustScaler** | $(x - Q_2) / (Q_3 - Q_1)$ | **CON OUTLIERS** | Robusto a outliers |

### 🏷️ Codificación Categórica:

| Método | Descripción | Cuándo usar | Dimensionalidad |
|--------|-------------|-------------|------------------|
| **Label Encoding** | Asigna números (0,1,2...) | Variables ordinales, target, árboles | No aumenta |
| **Ordinal Encoding** | Asigna números con orden | Variables ordinales con orden específico | No aumenta |
| **One-Hot Encoding** | Crea columna binaria por categoría | Variables nominales, modelos lineales | Aumenta (n-1 o n) |

### ✅ Reglas de Oro:

1. **SIEMPRE** usa `fit_transform()` en datos de entrenamiento
2. **SIEMPRE** usa solo `transform()` en datos de prueba
3. **Usa pipelines** para evitar errores y fuga de datos
4. **RobustScaler** es tu amigo cuando hay outliers
5. **One-Hot** para nominales, **Ordinal** para ordinales
6. **Escala DESPUÉS de dividir** train/test
7. **Guarda el scaler/encoder** para usar en producción

## 18. Ejercicios Prácticos

### Ejercicio 1:
Compara el rendimiento de un modelo con diferentes combinaciones de escaladores y encoders. ¿Cuál da mejor resultado?

### Ejercicio 2:
Crea un transformador personalizado que:
- Detecte automáticamente outliers
- Aplique el escalador apropiado según la presencia de outliers

### Ejercicio 3:
Implementa Target Encoding para variables categóricas con alta cardinalidad y compara con One-Hot Encoding.

### Ejercicio 4:
Analiza el impacto de `drop='first'` vs `drop=None` en One-Hot Encoding para prevenir multicolinealidad.

In [ ]:
# Espacio para tus ejercicios